In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_dir = '../data/'

### Countries 
https://www.countries-ofthe-world.com/countries-of-africa.html 

In [ ]:
countries_raw = '''A
Algeria
Angola
B
Benin
Botswana
Burkina Faso
Burundi
C
Cabo Verde
Cameroon
Central African Republic (CAR)
Chad
Comoros
Congo, Democratic Republic of the
Congo, Republic of the
Cote d'Ivoire
D
Djibouti
E
Egypt
Equatorial Guinea
Eritrea
Eswatini (formerly Swaziland)
Ethiopia
G
Gabon
Gambia
Ghana
Guinea
Guinea-Bissau
K
Kenya
L
Lesotho
Liberia
Libya
M
Madagascar
Malawi
Mali
Mauritania
Mauritius
Morocco
Mozambique
N
Namibia
Niger
Nigeria
R
Rwanda
S
Sao Tome and Principe
Senegal
Seychelles
Sierra Leone
Somalia
South Africa
South Sudan
Sudan
T
Tanzania
Togo
Tunisia
U
Uganda
Z
Zambia
Zimbabwe
'''

In [ ]:
countries = [c for c in countries_raw.split('\n') if len(c) > 1]
countries

In [ ]:
for c in countries:
    print(c)

In [ ]:
def is_country_in_str(s):
    pos = []
    for c in countries:
        if c in s:
            pos.append(c)
        
    if len(pos) > 0:
        lengths = list(map(lambda c: len(c), pos))
        c = pos[np.argmax(lengths)]
        return True, c
    else:
        return False, ''

### Population 

#### download 

In [ ]:
population_file_raw = 'population_africa_wikipedia_raw.html'

In [ ]:
!curl https://en.wikipedia.org/wiki/List_of_African_countries_by_population > {data_dir + population_file_raw}

#### parse 

In [ ]:
with open(data_dir + population_file_raw, 'rb') as fo:
    population_raw = str(fo.read())

In [ ]:
population_raw = [line for line in population_raw.split('<td style="text-align:left;">') if is_country_in_str(line)[0]]
population_raw

In [ ]:
def parse_population_cell(blob, verbose=False):
    data_raw = blob.split('><')[-1].split('\\n')
    if verbose: print(data_raw)
    p_str = data_raw[1].split('>')[-1].replace(',','')
    u_str = data_raw[-2].replace('<td>', '').replace('</td>', '')
    
    if verbose: print(p_str, u_str)
    population = int(p_str)
    return population, u_str

In [ ]:
parsed_population = {}
for blob in population_raw:
    c = is_country_in_str(blob)[1]
    population, update = parse_population_cell(blob, True)
    parsed_population[c] = dict(population=population, update=update)


In [ ]:
blob

In [ ]:
c, population, update

In [ ]:
parsed_population